In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import os
import json 
import timm

import torch
from torch import nn
from torchvision import models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A

# Label

In [2]:
# 변수 설명 csv 파일 참조
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

def label_preprocessing(path) :
    labels = pd.read_csv(path)

    cnt = 0
    label_encoder = {}
    for i, label in enumerate(tqdm(sorted(labels['label']))) :
        
        if label not in label_encoder.values() :
            label_encoder[cnt] = label
            cnt += 1
        
    label_decoder = {val : key for key, val in label_encoder.items()}
    
    return label_encoder, label_decoder

# CSV feature - min, max value 

In [14]:
def csv_feature_dict(path, csv_features) :
    
    csv_files = sorted(glob(os.path.join(path, '*/*.csv')))

    temp_csv = pd.read_csv(csv_files[0])[csv_features]
    max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

    # feature 별 최대값, 최솟값 계산
    for csv in tqdm(csv_files[1:]):
        temp_csv = pd.read_csv(csv)[csv_features]
        temp_csv = temp_csv.replace('-',np.nan).dropna()
        if len(temp_csv) == 0:
            continue
        temp_csv = temp_csv.astype(float)
        temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
        max_arr = np.max([max_arr,temp_max], axis=0)
        min_arr = np.min([min_arr,temp_min], axis=0)

    # feature 별 최대값, 최솟값 dictionary return
    return {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}

In [15]:
csv_feature_dict = csv_feature_dict('../data/train', csv_features)
csv_feature_dict

100%|███████████████████████████████████████████████████████████████████████████████| 5766/5766 [02:26<00:00, 39.40it/s]


{'내부 온도 1 평균': [3.4, 47.3],
 '내부 온도 1 최고': [3.4, 47.6],
 '내부 온도 1 최저': [3.3, 47.0],
 '내부 습도 1 평균': [23.7, 100.0],
 '내부 습도 1 최고': [25.9, 100.0],
 '내부 습도 1 최저': [0.0, 100.0],
 '내부 이슬점 평균': [0.1, 34.5],
 '내부 이슬점 최고': [0.2, 34.7],
 '내부 이슬점 최저': [0.0, 34.4]}

In [71]:
csv_path = '../data/train/10037/10037.csv'
df = pd.read_csv(csv_path)[csv_feature_dict.keys()]
df = df.replace('-', 0)

# MinMax scaling
for col in df.columns:
    df[col] = df[col].astype(float) - csv_feature_dict[col][0]
    df[col] = df[col] / (csv_feature_dict[col][1]-csv_feature_dict[col][0])

display(len(df))

df_dict = {}
for key in csv_feature_dict.keys() :
    df_dict[key] = np.max(df[key].to_numpy(), -1)

after_df = df.to_numpy()
df_len, df_features = after_df.shape

display(df.to_numpy().shape)
x        = np.zeros([512, df_features])
display(x.shape)


294

(294, 9)

(512, 9)

# Custom Dataset 

In [72]:
class CustomDataset(Dataset):
    def __init__(self, 
                 files, 
                 transforms, 
                 label_encoder, 
                 csv_feature_dict, 
                 max_len,
                 num_features,
                 mode='train'):
        
        self.mode = mode
        self.files = files
        self.label_encoder = label_encoder #label_encoder
        self.csv_feature_dict = csv_feature_dict
        self.max_len = max_len
        self.transforms = transforms
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('\\')[-1]
        
        # CSV
        csv_data = csv_preprocessing(file, file_name)
        
        # image
        img = img_preprocessing(file, file_name)
        
        if self.mode == 'train':         
            # Label
            label = label_preprocessing(file, file_name)
            
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'label' : torch.tensor(self.label_encoder[label], dtype=torch.long),
                'csv_feature': torch.tensor(x, dtype=torch.float32)
            }
        
        else:
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'csv_feature': torch.tensor(x, dtype=torch.float32)
            }

    def csv_preprocessing(self, file, file_name) :
        # CSV
        csv_path = f'{file}/{file_name}.csv'
        df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
        df = df.replace('-', 0)
        
        # MinMax scaling
        for col in df.columns:
            df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
            df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])

        seq_len = len(df)
        df_np = df.to_numpy()
        df_len, df_features = df_np.shape
        
        csv_data = np.zeros([self.max_len, df_features])
        csv_data[0:df_len, :] = df_np
        
        return csv_data

    def img_preprocessing(self, file, file_name) :
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(image=img)["image"]
        img = img.transpose(2,0,1)
        
        return img
    
    def label_preprocessing(self, file, file_name) :
        json_path = f'{file}/{file_name}.json'
        with open(json_path, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        
        return f'{crop}_{disease}_{risk}'

# Model - CNN

In [130]:
class CNN_Encoder(nn.Module):
    def __init__(self, model_name, num_classes, pretrained_path=None):
        super(CNN_Encoder, self).__init__()
        
        if pretrained_path :
            # no use pretrained model trained with Public dataset
            self.model = self.create_pretrained_model(model_name, num_classes, pretrained_path)
            
        else :            
            self.model = timm.create_model(model_name, num_classes=num_classes, pretrained=True)
            
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

    def create_pretrained_model(self, model_name, num_classes, pretrained_path):
        pre_model = torch.load(pretrained_path, map_location="cpu")
        output_size = pre_model[list(pre_model.keys())[-1]].shape[0]
 
        return nn.Sequential(
                    timm.create_model(model_name, num_classes=output_size, pretrained=True),
                    nn.Linear(output_size, num_classes)
                )

# model = CNN_Encoder("efficientnetv2_rw_s", 1000)
# model = CNN_Encoder("efficientnetv2_rw_s", 1000, "../model/k_fold_50k_pretrained_effiv2S/4_f9462_public_vill_50k_pretrain_efficientnetv2S.pt")

# Model - RNN


In [131]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class MLSTMfcn(nn.Module):
    def __init__(self, *, num_classes, max_seq_len, num_features,
                 num_lstm_out=128, num_lstm_layers=1, 
                 conv1_nf=128, conv2_nf=256, conv3_nf=128,
                 lstm_drop_p=0.8, fc_drop_p=0.3, cnn_out=1000):
        
        super(MLSTMfcn, self).__init__()
        self.num_classes = num_classes
        self.max_seq_len = max_seq_len
        self.num_features = num_features

        self.num_lstm_out = num_lstm_out
        self.num_lstm_layers = num_lstm_layers

        self.conv1_nf = conv1_nf
        self.conv2_nf = conv2_nf
        self.conv3_nf = conv3_nf

        self.lstm_drop_p = lstm_drop_p
        self.fc_drop_p = fc_drop_p

        self.lstm = nn.LSTM(input_size=self.num_features, 
                            hidden_size=self.num_lstm_out,
                            num_layers=self.num_lstm_layers,
                            batch_first=True)
        
        self.conv1 = nn.Conv1d(self.num_features, self.conv1_nf, 8)
        self.conv2 = nn.Conv1d(self.conv1_nf, self.conv2_nf, 5)
        self.conv3 = nn.Conv1d(self.conv2_nf, self.conv3_nf, 3)

        self.bn1 = nn.BatchNorm1d(self.conv1_nf)
        self.bn2 = nn.BatchNorm1d(self.conv2_nf)
        self.bn3 = nn.BatchNorm1d(self.conv3_nf)

        self.se1 = SELayer(self.conv1_nf)  # ex 128
        self.se2 = SELayer(self.conv2_nf)  # ex 256

        self.relu = nn.ReLU()
        self.lstmDrop = nn.Dropout(self.lstm_drop_p)
        self.convDrop = nn.Dropout(self.fc_drop_p)

        self.fc = nn.Linear(self.conv3_nf+self.num_lstm_out, 128)

        self.out_layer = nn.Linear(cnn_out+128, self.num_classes)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, enc_out, x, seq_lens):
        ''' input x should be in size [B,T,F], where 
            B = Batch size
            T = Time samples
            F = features
        '''
        x1 = nn.utils.rnn.pack_padded_sequence(x, seq_lens.cpu(), 
                                               batch_first=True, 
                                               enforce_sorted=False)
        x1, (ht,ct) = self.lstm(x1)
        x1, _ = nn.utils.rnn.pad_packed_sequence(x1, batch_first=True, 
                                                 padding_value=0.0)
        x1 = x1[:,-1,:]
        
        x2 = x.transpose(2,1)
        x2 = self.convDrop(self.relu(self.bn1(self.conv1(x2))))
        x2 = self.se1(x2)
        x2 = self.convDrop(self.relu(self.bn2(self.conv2(x2))))
        x2 = self.se2(x2)
        x2 = self.convDrop(self.relu(self.bn3(self.conv3(x2))))
        x2 = torch.mean(x2,2)
        
        x_all = torch.cat((x1,x2),dim=1)
        x_out = self.fc(x_all)
        concat = torch.cat([enc_out, x_out], dim=1)  # enc_out + hidden 
        output = self.dropout(concat)
        x_output = self.out_layer(output)
        x_out = F.log_softmax(x_output, dim=1)

        return x_out
    
# model = MLSTMfcn(num_classes=38, max_seq_len=512, num_features=9)

# Model - CNN + RNN

In [133]:
class CNN2RNN(nn.Module):
    def __init__(
        self,
        model_name,
        num_classes,
        max_len,
        num_features,
        pretrained_path=None,
    ):
        super(CNN2RNN, self).__init__()
        
        self.cnn = CNN_Encoder(model_name, num_classes, pretrained_path)
        self.rnn = MLSTMfcn(num_classes=num_classes, max_seq_len=max_len, num_features=num_features)


    def forward(self, img, seq, seq_len):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq, seq_len)
        
        return output
    
# model = CNN2RNN("efficientnetv2_rw_s", 
#                 1000, 
#                 512,
#                 6,
#                 "../model/k_fold_50k_pretrained_effiv2S/4_f9462_public_vill_50k_pretrain_efficientnetv2S.pt")
# model